# Julia's Type System
The biggest difference between Julia and Matlab (and to some degree Python) is its type system which is much richer, and allows the user fine control over how his data is structured, and how his program is executed.

Variables can optionally be given type annotations:

In [1]:
a::Int = 1
b::Float64 = 0.2
c::Vector{Int} = [1,2,3]

3-element Vector{Int64}:
 1
 2
 3

New types can be defined via structs

In [2]:
struct Student
    name::String
    age::Int
    section::String
    year::Int
end

In [3]:
alice = Student("Alice", 22, "Chemistry", 4)

Student("Alice", 22, "Chemistry", 4)

In [4]:
alice.section

"Chemistry"

By default, structs are *immutable*, so we get an error when trying to modify a field

In [5]:
alice.section = "Physics"

ErrorException: setfield!: immutable struct of type Student cannot be changed

If we want to be able to mutate a struct's fields, we need to declare it as mutable

In [6]:
mutable struct Lightbulb
    age::Int
    wattage::Float64
    turned_on::Bool
end

In [7]:
lightbulb = Lightbulb(0, 150.0, false)

Lightbulb(0, 150.0, false)

In [8]:
lightbulb.turned_on = true
lightbulb

Lightbulb(0, 150.0, true)

## Parametric Types
Sometimes, we want to be flexible with the types inside our structs. That's where type parameters come in. The following cell defines a new type that has two fields whose type we can choose

In [9]:
struct Point2D{T}
    x::T
    y::T
end

In [10]:
Point2D(x) = Point2D(x,x)

Point2D

In [11]:
Point2D(1)

Point2D{Int64}(1, 1)

In [12]:
p1 = Point2D(1,2)

Point2D{Int64}(1, 2)

In [13]:
p2 = Point2D(1.0, 2.0)

Point2D{Float64}(1.0, 2.0)

## Subtyping

In addition to parametric types, there is also a partial order (type hierarchy). This is similar to object-oriented programming, with one main difference.

All the types we've seen so far were *concrete types*, that is types that have a representation in memory. There also *abstract types*, which don't have fields or a representation, and can't actually be instantiated at runtime.

Concrete types can only have a supertype and cannot have subtypes, whereas abstract types can have arbitrarily many subtypes.

Since julia types are first class citizens, we can evaluate the order relation at runtime, for instance to check that `Int` is a subtype of the abstract type `Integer`:

In [14]:
Int <: Integer

true

When declaring a new type, we can specify its immediate supertype:

In [1]:
abstract type Animal end
abstract type Mammal <: Animal end
struct Tiger <: Mammal end
struct Dog <: Mammal end

## Types and Functions

Function arguments can optionally be type annotated. In most cases this isn't necessary, however, and is only recommended when strictly needed.

In [16]:
function collatz(n::Int)
    iseven(n) ? div(n,2) : 3n+1
end

collatz (generic function with 1 method)

It's also possible to specify a "return" type. This only means that the result of the function gets converted to that type, so this is not a formal garrantee.

In [17]:
function collatz(n::Int)::Int
    iseven(n) ? div(n,2) : 3n+1
end

collatz (generic function with 1 method)

## Multiple Dispatch
Where the type system really comes together is in how it interacts when defining multiple versions of the same function with different argument types. These are called methods, and allow an extreme degree of modularity in Julia code.

As an example, let's go back to our `Animal` type, and add a couple more species.

In [2]:
abstract type Bird <: Animal end
struct Parrot <: Bird end
struct Crow <: Bird end

Let's say we want to write a function that returns the sound an animal makes. Since we're lazy, we don't want to have to write a method for every single type. For now, we'll say that by default, mammals growl and birds caw. We can do that by writing methods for the abstract types `Mammal` and `Bird`.

In [3]:
sound(::Mammal) = "Growl"
sound(::Bird) = "Caw"

sound (generic function with 2 methods)

In [8]:
sound(Tiger())

"Growl"

In [5]:
sound(Parrot())

"Caw"

Now let's say we want to add cows to our growing list of animals. Obviously cows don't growl, so we can write a method for `sound(::Cow)` while leaving other mammals unchanged.

In [6]:
struct Cow <: Mammal end
sound(::Cow) = "Moo"

sound (generic function with 3 methods)

In [7]:
sound(Cow())

"Moo"

## Dispatching over multiple arguments
So far we've used dispatch for functions with a single argument. This is equivalent to *single dispatch*, which is how Objected Oriented languages like Python specialize methods. Julia however can specialize methods over the types of multiple arguments at the same time.

To illustrate this, let's model chemical reactions.

In [9]:
abstract type Chemical end
struct H2 <: Chemical end
struct H2O <: Chemical end
struct O2 <: Chemical end
struct SO3 <: Chemical end

We want to write a function that tells us what happens when we mix two chemicals. Now for the chemicals we've implemented, there's really only two interesting cases:
1. We mix Hydrogen (`H2`) and Oxygen (`O2`) to get water (and more importantly a small explosion)
2. We mix water (`H2O`) and sulfur trioxide (`SO3`) to get sulfuric acid

Any other combination of chemicals won't do anything.

A naive way to implement this would be the following

In [10]:
function mix_bad(x,y)
    # `x isa T` returns true if x is of type T (or a subtype of T)
    if x isa H2 && y isa O2
        return "Boom!"
    elseif x isa H2O && y isa SO3
        return "Sulfuric Acid!"
    else
        return "Nothing happens."
    end
end

mix_bad (generic function with 1 method)

In [11]:
mix_bad(H2(), O2())

"Boom!"

In [12]:
mix_bad(O2(), H2())

"Nothing happens."

Why is this bad? Well, to begin with, if we wanted to add more chemicals, we'd have to add more clauses to that if, which could quickly become enormous. But then, if we distributed this function in a package, other people wouldn't be able to add their own chemicals.

Another reason is that we're comparing types at runtime, instead on relying on the compiler to optimize our code, which means this function will be less performant.

The correct way to do this, using multiple dispatch is as follows

In [13]:
mix_good(::H2, ::O2) = "Boom!"
mix_good(::O2, ::H2) = "Boom!"
mix_good(::H2O, ::SO3) = "Sulfuric Acid!"
mix_good(::SO3, ::H2O) = "Sulfuric Acid!"
mix_good(::Chemical, ::Chemical) = "Nothing happens."

mix_good (generic function with 5 methods)

For another example: https://www.moll.dev/projects/effective-multi-dispatch/

## Advanced Example: Automatic Differentiation
We'll finish with a more complicated example to showcase how powerful Multiple dispatch can be, namely, we'll implement a barebones way of *automatically* computing the derivative of arithmetic operations in about ten lines of code.

This example is largely adapted from https://www.youtube.com/watch?v=vAp6nUMrKYg

Let's start by defining a new type called `Dual`. It represents pairs of numbers of the form 
$$ x + \epsilon y $$
where $\epsilon$ is a bit like the imaginary unit $i$, except it satisfies $\epsilon^2 = 0$.

Another way to interpret them is that $x$ represents the value of some function evaluated at some point, while $y$ represents the value of the *derivative* of that function at the same point.

In [14]:
struct Dual <: Number
    x::Float64
    y::Float64
end

Note that we could make this a parametric type, but won't for clarity's sake.

We've defined `Dual` as a subtype of `Number` which is the abstract type for objects that support arithmetic operations. Let's define them for our new type.

In [15]:
import Base: +, -, *, /, convert, promote_rule, show

# The following four lines define arithmetic operations on Duals. Notice how the rules for * and / are the same as derivative rules for multiplication and division
+(a::Dual, b::Dual) = Dual(a.x + b.x, a.y + b.y)
-(a::Dual, b::Dual) = Dual(a.x - b.x, a.y - b.y)
*(a::Dual, b::Dual) = Dual(a.x*b.x, a.y*b.x + a.x*b.y)
/(a::Dual, b::Dual) = Dual(a.x/b.x, (a.y*b.x - a.x*b.y)/(b.x^2))

# These three lines tell how to convert between real numbers and Dual, and how to pretty print Dual numbers.
convert(::Type{Dual}, x::Real) = Dual(x, zero(x))
promote_rule(::Type{Dual}, ::Type{<:Number}) = Dual
show(io::IO, d::Dual) = print(io, d.x, " + ", d.y," ϵ")

show (generic function with 333 methods)

We can try it out on some function to check that it works

In [16]:
f(x) = 2*x^2 + x
f(1), f(1.0), f(Dual(1,1))

(3, 3.0, 3.0 + 5.0 ϵ)

So far, so good. Let's try a more complicated function.

In [17]:
"""
Compute the square root of x using the Babylonian algorithm
"""
function babylonian(x; nmax = 10)
    t = (1+x)/2
    for i in 2:nmax
        t = (t + x/t)/2
    end
    return t
end

babylonian

In [18]:
babylonian(π), sqrt(π), babylonian(2), sqrt(2)

(1.7724538509055159, 1.7724538509055159, 1.414213562373095, 1.4142135623730951)

In [19]:
x = Dual(2, 1)
babylonian(x)

1.414213562373095 + 0.35355339059327373 ϵ

In [20]:
0.5/√2

0.35355339059327373